In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/housesalesprediction/kc_house_data.csv')
df['Current_Year'] = 2021
df['years_old'] = df['Current_Year'] - df['yr_built']
df.drop(['Current_Year'], axis = 1 , inplace = True)
df.drop(['yr_built'], axis = 1 , inplace = True)
df.drop(['id'], axis = 1 , inplace = True)
df.drop(['date'], axis = 1 , inplace = True)

df.shape
# df.head()



In [ ]:
#To get a dataframe having only a particular value in a particular column 
# and the rest columns correspond to those values

# Here the values are 0 and 1 ,and the column is waterfront

x = df[df['waterfront'] == 1].index.tolist()
y = df[df['waterfront'] == 0].index.tolist()
# print(len(x))
# print(len(y))
df.columns
df.head()

In [ ]:
# df1 is a DF having no rows in which bedrooms == 0
df1 = df[df['bedrooms'] != 0]

# df2 is a DF having no rows in which bathrooms == 0
df2 = df1[df1['bathrooms'] != 0]
print(len(df2))
df2.shape



In [ ]:
df2.corr()

In [ ]:
#Fixing a possible typo 
print(df2.loc[df2['bedrooms']== 33])
15870

In [ ]:
#fixed an anamoly having 33 bedrooms(considering it a typing error)
df2.at[15870,'bedrooms'] = 3
# print(df2.loc[df2['price']== 640000])

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,10))
df_corr = df2.drop(['lat','long','zipcode'], axis = 1)
df_corr = df_corr.corr()
mask = np.triu(df_corr.corr())
sns.heatmap(df_corr, annot=True, mask=mask, square=True, 
            fmt='.1g',  
            vmin=-1, vmax=1, center= 0, cmap='coolwarm',
            linewidths=3, linecolor='black',
            cbar_kws= {'orientation': 'vertical'}) 




The highest correlation between the target and the independent variables:

sqft_living;
sqft_above;
grade;
sqft_living15.

In [ ]:
z = df.loc[df['yr_renovated'] == 0]
print(z)

In [ ]:
#Most of the values in yr_renovated are 0s
list1 = []
for x in df2['yr_renovated']:
    if x == 0:
        list_.append(0)
    else:
        list_.append(1)
        
df2['renovated'] = list1
df2.head()

    


In [ ]:
z = df2.loc[df2['renovated'] == 0]
print(z)

In [ ]:
df2.corr()
df2.head()

In [ ]:
figure = plt.figure()
axes1 = figure.add_axes([1.2, -3.9, 1, 1]) 
sns.regplot(x='lat', y='price', data=df2, color='royalblue', ax=axes1, scatter_kws={'s' : 10}, line_kws={'color': 'crimson', 'lw': 5})
axes1.set_title('Price depending of latitude', fontsize=17)
axes1.set_xlabel('lat', fontsize=14)
axes1.set_ylabel('price', fontsize=14)

axes2 = figure.add_axes([2.4, -3.9, 1, 1]) 
sns.regplot(x='long', y='price', data=df2, color='royalblue', ax=axes2, scatter_kws={'s' : 10}, line_kws={'color': 'crimson', 'lw': 5})
axes2.set_title('Price depending of longitude', fontsize=17)
axes2.set_xlabel('long', fontsize=14)
axes2.set_ylabel('price', fontsize=14)

In [ ]:
from sklearn.preprocessing import StandardScaler


columns = list(df2.columns)
scaler = StandardScaler()
df_st = scaler.fit_transform(df2)
df_st = pd.DataFrame(data=df_st, columns=columns)
print(columns)




In [ ]:

df_st.drop(['lat'], axis = 1, inplace = True)
df_st.drop(['long'], axis = 1 , inplace = True)
df_st.head()

In [ ]:
df_st.head()

In [ ]:
df_st.mean(axis = 0)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
x = df_st.drop(['price'], axis=1)
y = df_st['price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

params1 = {}
linear = GridSearchCV(LinearRegression(),params1, cv = 5)
linear.fit(x_train, y_train)
y_predict_linear = linear.predict(x_test)
r_sq_linear = linear.score(x_test, y_test)

poly = GridSearchCV(LinearRegression(), params1, cv = 5)
poly.fit(x_train_poly, y_train)
y_predict_poly = poly.predict(x_test_poly)
r_sq_poly = poly.score(x_test_poly, y_test)

params2 = {'alpha':[0.001, 0.01, 0.1, 1, 10, 100]}
ridge = GridSearchCV(Ridge(), params2, cv=5)
ridge.fit(x_train_poly, y_train)
y_predict_ridge = ridge.predict(x_test_poly)
r_sq_ridge = ridge.score(x_test_poly, y_test)

params3 = {'alpha':[0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
elastic = GridSearchCV(ElasticNet(), params3, cv=5)
elastic.fit(x_train_poly, y_train)
y_predict_elastic = elastic.predict(x_test_poly)
r_sq_elastic = elastic.score(x_test_poly, y_test)







In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer

pipe = Pipeline((
     ('pt',PowerTransformer()),
     ('rf',RandomForestRegressor()),
))

pipe.fit(x_train,y_train)
rf_score=pipe.score(x_test,y_test)
print('Rf score :', rf_score)



params2 = {'alpha':[0.001, 0.01, 0.1, 1, 10, 100]}
lasso = GridSearchCV(Lasso(), params2, cv=5)
lasso.fit(x_train_poly, y_train)
y_predict_lasso = lasso.predict(x_test_poly)
r_sq_lasso = lasso.score(x_test_poly, y_test)

models = ['RandomForest','Linear','Poly','Ridge','Lasso','Elastic']
r_sq = [rf_score,r_sq_linear,r_sq_poly,r_sq_ridge,r_sq_lasso,r_sq_elastic]
r_sq_table = pd.DataFrame({'Model' : models , 'r^2' : r_sq})
r_sq_table.sort_values( by = 'r^2' , axis = 0 , ascending = False)